In [1]:
#refer https://www.kaggle.com/waxbabi/light-gbm-benchmark-0-3692/code
#implement basic content-based prediction

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

In [2]:
products = pd.read_csv('../csv/products.csv')

In [3]:
order_products_prior = pd.read_csv('../csv/order_products__prior.csv')

In [4]:
order_products_train = pd.read_csv('../csv/order_products__train.csv')

In [5]:
orders = pd.read_csv('../csv/orders.csv')

In [6]:
df = order_products_prior.set_index('order_id').join(orders.set_index('order_id'))

In [7]:
product_user = df.groupby(['product_id', 'user_id']).agg({'reordered': ['sum', 'count'], 
'order_number': ['mean'], 'days_since_prior_order': ['mean'], 'add_to_cart_order': ['mean']})\
.reset_index().set_index(['product_id', 'user_id']).add_prefix('pu')

In [8]:
pu = pd.DataFrame(df.groupby(['product_id', 'user_id']).size().rename('count')).reset_index()

In [9]:
truncted_product_user = pu.copy()
truncted_product_user.loc[truncted_product_user['count'] > 10, 'count'] = 10

In [10]:
product_level = pd.pivot_table(truncted_product_user, index=['product_id'], columns=['count'], aggfunc=len, fill_value=0)\
.reset_index().set_index('product_id').add_prefix('plevel_')

In [11]:
user_level = pd.pivot_table(truncted_product_user, index=['user_id'], columns=['count'], aggfunc=len, fill_value=0)\
.reset_index().set_index('user_id').add_prefix('ulevel_')

In [12]:
product_v = df.reset_index().groupby('product_id').agg({'order_id': ['nunique'], 'user_id': ['nunique'], 
'reordered': ['sum', 'count'], 'add_to_cart_order': ['mean'], 'days_since_prior_order': ['mean']})\
.reset_index().set_index('product_id').add_prefix('p_')

In [13]:
user_v = df.reset_index().groupby('user_id').agg({'days_since_prior_order': ['mean'], 'order_id': ['nunique'], 
'product_id': ['nunique'], 'reordered': ['sum', 'count']})\
.reset_index().set_index('user_id').add_prefix('u_')

In [14]:
label = orders[orders.eval_set == 'train'].set_index('order_id')\
.join(order_products_train.set_index('order_id'))[['product_id', 'user_id']].set_index(['product_id', 'user_id'])
label['label'] = 1

In [ ]:
train = orders[orders.eval_set == 'train'].set_index('user_id')\
.join(product_user.reset_index().set_index('user_id'))\
.rename(index=str, columns={('product_id', ''): 'product_id'})\


In [32]:
train = orders[orders.eval_set == 'train'].set_index('user_id')\
.join(product_user.reset_index().set_index('user_id'))\
.rename(index=str, columns={('product_id', ''): 'product_id'})\
.reset_index().set_index(['product_id', 'user_id']).join(label)\
.reset_index().set_index('product_id').join(product_level).join(product_v)\
.join(products[['product_id', 'aisle_id', 'department_id']].set_index('product_id'))\
.reset_index().set_index('user_id').join(user_level).join(user_v)\
.reset_index().fillna(0)

In [33]:
train.to_csv('../data/train.data')

In [35]:
test = orders[orders.eval_set == 'test'].set_index('user_id')\
.join(product_user.reset_index().set_index('user_id'))\
.rename(index=str, columns={('product_id', ''): 'product_id'})\
.reset_index().set_index('product_id').join(product_level).join(product_v)\
.join(products[['product_id', 'aisle_id', 'department_id']].set_index('product_id'))\
.reset_index().set_index('user_id').join(user_level).join(user_v)\
.reset_index().fillna(0)

In [36]:
test.to_csv('../data/test.data')